<a href="https://colab.research.google.com/github/netra-poonia/simulation1/blob/main/CO_CM_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `Credit Officer Base Calculation`

In [2]:
import pandas as pd
import numpy as np

In [4]:
def calculate_co_incentive(disbursal_df, employee_df, reporting_month):
    """
    Calculates the monthly incentive for Credit Officers based on TAT and Business performance.

    Args:
        disbursal_df (pd.DataFrame): DataFrame containing disbursal data.
        employee_df (pd.DataFrame): DataFrame containing employee details.
        reporting_month (str): The month for which to calculate incentive (e.g., 'August 2025').

    Returns:
        pd.DataFrame: A DataFrame with detailed incentive calculations for each Credit Officer.
    """
    print(f"--- Starting Incentive Calculation for {reporting_month} ---")

    # --- 1. Data Preparation and TAT Calculation ---

    # Filter disbursal data for the specified month
    monthly_disbursal_df = disbursal_df[disbursal_df['disb_month'] == reporting_month].copy()

    if monthly_disbursal_df.empty:
        print(f"No disbursal data found for {reporting_month}. Exiting.")
        return pd.DataFrame()

    # Calculate the median 'l2d_tat' for each state for the given month
    print("Calculating state-wise median Turn Around Time (TAT)...")
    state_median_tat = monthly_disbursal_df.groupby('state_name')['l2d_tat'].transform('median')
    monthly_disbursal_df['state_median_tat'] = state_median_tat

    # Flag applications that are within the defined state-wise TAT
    # 1 if TAT is met, 0 otherwise
    monthly_disbursal_df['in_defined_tat'] = (monthly_disbursal_df['l2d_tat'] <= monthly_disbursal_df['state_median_tat']).astype(int)

    # Merge employee data to get designation, targets, etc.
    print("Merging disbursal data with employee details...")
    final_df = pd.merge(monthly_disbursal_df, employee_df, on='co_code', how='left')


    # --- 2. TAT Score Calculation ---
    print("Calculating TAT Score for each Credit Officer...")

    # Group by Credit Officer to calculate performance metrics
    co_performance = final_df.groupby('co_code').agg(
        total_applications=('application_id', 'count'),
        applications_in_tat=('in_defined_tat', 'sum')
    ).reset_index()

    # Calculate the ratio of applications within the defined TAT
    co_performance['tat_ratio'] = co_performance['applications_in_tat'] / co_performance['total_applications']

    # Define the conditions and scores for TAT performance
    conditions_tat = [
        co_performance['tat_ratio'] > 0.70,
        (co_performance['tat_ratio'] > 0.50) & (co_performance['tat_ratio'] <= 0.70),
        co_performance['tat_ratio'] <= 0.50
    ]
    scores_tat = [100, 75, 50]

    # Assign TAT score based on the ratio
    co_performance['tat_score'] = np.select(conditions_tat, scores_tat, default=0)


    # --- 3. Business Score Calculation ---
    print("Calculating Business Score for each Credit Officer...")

    # Assign business points based on product type
    # Secured = 1 point, Unsecured = 0.25 points
    final_df['business_points'] = np.where(final_df['product_type'] == 'Secured', 1, 0.25)

    # Calculate total business points for each CO
    business_points_summary = final_df.groupby('co_code')['business_points'].sum().reset_index()
    co_performance = pd.merge(co_performance, business_points_summary, on='co_code', how='left')

    # Merge target information into the performance summary
    co_performance = pd.merge(co_performance, employee_df[['co_code', 'target_level']], on='co_code', how='left')

    # Define a function to calculate business score based on target level
    def get_business_score(row):
        points = row['business_points']
        target = row['target_level']
        if target == 1:
            if points > 5: return 100
            elif points > 3: return 75
            else: return 50
        elif target == 2:
            if points > 6: return 100
            elif points > 4: return 75
            else: return 50
        elif target == 3:
            if points > 9: return 100
            elif points > 5: return 75
            else: return 50
        elif target == 4:
            if points > 12: return 100
            elif points > 7: return 75
            else: return 50
        else:
            return 0 # Default score if target level is not defined

    # Apply the function to calculate business score
    co_performance['business_score'] = co_performance.apply(get_business_score, axis=1)


    # --- 4. Final Incentive Calculation ---
    print("Calculating final incentive...")

    # Calculate Overall Score (50% weightage for TAT, 50% for Business)
    co_performance['overall_score'] = (co_performance['tat_score'] * 0.5) + (co_performance['business_score'] * 0.5)

    # Merge salary information
    co_performance = pd.merge(co_performance, employee_df[['co_code', 'employee_name', 'branch', 'state_name', 'monthly_salary']], on='co_code', how='left')

    # Calculate eligible incentive (25% of monthly salary)
    co_performance['eligible_incentive'] = co_performance['monthly_salary'] * 0.25

    # Calculate the final incentive amount
    co_performance['final_incentive'] = co_performance['eligible_incentive'] * (co_performance['overall_score'] / 100)

    # Format the final incentive to two decimal places for readability
    co_performance['final_incentive'] = co_performance['final_incentive'].round(2)


    # --- 5. Final Report Generation ---
    print("Generating the final incentive report...")

    # Select and reorder columns for the final report for clarity and transparency
    final_report = co_performance[[
        'co_code',
        'employee_name',
        'branch',
        'state_name',
        'monthly_salary',
        'total_applications',
        'applications_in_tat',
        'tat_ratio',
        'tat_score',
        'business_points',
        'target_level',
        'business_score',
        'overall_score',
        'eligible_incentive',
        'final_incentive'
    ]]

    # Format the tat_ratio as a percentage
    final_report['tat_ratio'] = pd.Series(["{0:.2f}%".format(val * 100) for val in final_report['tat_ratio']], index = final_report.index)


    print("--- Incentive Calculation Complete ---")
    return final_report

## Testing

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# disbursal_df = pd.read_csv('your_disbursal_file_name.csv')
# employee_df = pd.read_csv('your_employee_file_name.csv')

In [ ]:
disbursal_data = {
    'application_id': range(1, 21),
    'co_code': ['CO101']*5 + ['CO102']*6 + ['CO103']*5 + ['CO104']*4,
    'disb_month': ['August 2025']*20,
    'state_name': ['Maharashtra']*5 + ['Gujarat']*6 + ['Maharashtra']*5 + ['Gujarat']*4,
    'product_type': ['Secured', 'Unsecured', 'Secured', 'Secured', 'Unsecured',
                     'Secured', 'Unsecured', 'Unsecured', 'Unsecured', 'Unsecured', 'Secured',
                     'Secured', 'Secured', 'Secured', 'Secured', 'Secured',
                     'Unsecured', 'Unsecured', 'Unsecured', 'Unsecured'],
    'l2d_tat': [2, 5, 3, 4, 6,  # CO101 (Maharashtra)
                7, 9, 8, 10, 11, 6, # CO102 (Gujarat)
                1, 2, 3, 4, 5,  # CO103 (Maharashtra)
                9, 10, 11, 12] # CO104 (Gujarat)
}
disbursal_df = pd.DataFrame(disbursal_data)

# Sample Employee Data
employee_data = {
    'co_code': ['CO101', 'CO102', 'CO103', 'CO104'],
    'employee_name': ['Amit Sharma', 'Priya Singh', 'Rahul Verma', 'Sneha Patel'],
    'designation': ['Credit Officer', 'Credit Officer', 'Credit Officer', 'Credit Officer'],
    'branch': ['Mumbai', 'Ahmedabad', 'Pune', 'Surat'],
    'state_name': ['Maharashtra', 'Gujarat', 'Maharashtra', 'Gujarat'],
    'monthly_salary': [40000, 42000, 38000, 41000],
    'target_level': [1, 2, 3, 1] # Represents 1st month, 2nd month, etc.
}
employee_df = pd.DataFrame(employee_data)

# --- Execution ---
# Set the reporting month
reporting_month = 'August 2025'

# Calculate the incentive
incentive_report = calculate_co_incentive(disbursal_df, employee_df, reporting_month)

# Display the final report
if not incentive_report.empty:
    print("\n--- Credit Officer Incentive Report ---")
    print(incentive_report.to_string())

## `Credit Manager Base Calculation`

In [6]:
import pandas as pd
import numpy as np

def calculate_cm_incentive(disbursal_df, employee_df, reporting_months):
    """
    Calculates the quarterly incentive for Credit Managers based on TAT, Business, and Portfolio performance.

    Args:
        disbursal_df (pd.DataFrame): DataFrame containing disbursal data.
        employee_df (pd.DataFrame): DataFrame containing employee details for managers.
        reporting_months (list): A list of the three months in the quarter (e.g., ['July 2025', 'August 2025', 'September 2025']).

    Returns:
        pd.DataFrame: A DataFrame with detailed incentive calculations for each Credit Manager.
    """
    quarter_str = ", ".join(reporting_months)
    print(f"--- Starting Incentive Calculation for Quarter: {quarter_str} ---")

    # --- 1. Data Preparation ---

    # Filter disbursal data for the specified quarter
    quarterly_disbursal_df = disbursal_df[disbursal_df['disb_month'].isin(reporting_months)].copy()

    if quarterly_disbursal_df.empty:
        print(f"No disbursal data found for the specified quarter. Exiting.")
        return pd.DataFrame()

    # Merge with employee data to link disbursals to a Credit Manager via branch
    # Note: We assume one CM per branch in this model.
    quarterly_disbursal_df = pd.merge(quarterly_disbursal_df, employee_df[['cm_code', 'branch']], on='branch', how='left')

    # --- 2. TAT Score Calculation (Total Weightage: 30%) ---
    print("Calculating TAT Score for each Branch/Credit Manager...")

    # -- 2a. Login to Disbursement (L2D) TAT Component (15% Weightage) --
    state_median_l2d_tat = quarterly_disbursal_df.groupby('state_name')['l2d_tat'].transform('median')
    quarterly_disbursal_df['in_defined_l2d_tat'] = (quarterly_disbursal_df['l2d_tat'] <= state_median_l2d_tat).astype(int)

    # -- 2b. Login to Sanction (L2S) TAT Component (15% Weightage) --
    state_median_l2s_tat = quarterly_disbursal_df.groupby('state_name')['l2s_tat'].transform('median')
    quarterly_disbursal_df['in_defined_l2s_tat'] = (quarterly_disbursal_df['l2s_tat'] <= state_median_l2s_tat).astype(int)

    # Group by Credit Manager to calculate performance metrics
    cm_performance = quarterly_disbursal_df.groupby('cm_code').agg(
        total_applications=('application_id', 'count'),
        applications_in_l2d_tat=('in_defined_l2d_tat', 'sum'),
        applications_in_l2s_tat=('in_defined_l2s_tat', 'sum')
    ).reset_index()

    # Calculate TAT ratios
    cm_performance['l2d_tat_ratio'] = cm_performance['applications_in_l2d_tat'] / cm_performance['total_applications']
    cm_performance['l2s_tat_ratio'] = cm_performance['applications_in_l2s_tat'] / cm_performance['total_applications']

    # Define the scoring function for TAT components
    def get_tat_score(ratio):
        if ratio > 0.70:
            return 100
        elif ratio > 0.50:
            return 75
        else:
            return 50

    # Calculate scores for each TAT component
    cm_performance['l2d_tat_score'] = cm_performance['l2d_tat_ratio'].apply(get_tat_score)
    cm_performance['l2s_tat_score'] = cm_performance['l2s_tat_ratio'].apply(get_tat_score)

    # The final TAT score is an average of the two component scores
    cm_performance['final_tat_score'] = (cm_performance['l2d_tat_score'] + cm_performance['l2s_tat_score']) / 2

    # --- 3. Business Score Calculation (Total Weightage: 30%) ---
    print("Calculating Business Score...")

    # Business score is based on the total number of units (applications) disbursed in the branch
    # The 'total_applications' column is already calculated in the cm_performance df.

    # Define conditions and scores for Business performance
    conditions_business = [
        cm_performance['total_applications'] > 20,
        (cm_performance['total_applications'] > 12) & (cm_performance['total_applications'] <= 20),
        cm_performance['total_applications'] <= 12
    ]
    scores_business = [100, 75, 50]
    cm_performance['business_score'] = np.select(conditions_business, scores_business, default=0)

    # --- 4. Portfolio Score (Total Weightage: 40%) ---
    print("Applying placeholder for Portfolio Score...")
    # As per the requirement, data is not yet available for this component.
    # We will add a placeholder score of 0.
    # THIS SECTION SHOULD BE UPDATED ONCE PORTFOLIO METRICS ARE FINALIZED.
    cm_performance['portfolio_score'] = 0


    # --- 5. Final Incentive Calculation ---
    print("Calculating final incentive...")

    # Calculate Overall Score with specified weightages
    cm_performance['overall_score'] = (cm_performance['final_tat_score'] * 0.30) + \
                                      (cm_performance['business_score'] * 0.30) + \
                                      (cm_performance['portfolio_score'] * 0.40)

    # Merge employee details for salary and personal info
    cm_performance = pd.merge(cm_performance, employee_df, on='cm_code', how='left')

    # Calculate eligible incentive (assuming 25% of QUARTERLY salary)
    cm_performance['quarterly_salary'] = cm_performance['monthly_salary'] * 3
    cm_performance['eligible_incentive'] = cm_performance['quarterly_salary'] * 0.25

    # Calculate the final incentive amount
    cm_performance['final_incentive'] = cm_performance['eligible_incentive'] * (cm_performance['overall_score'] / 100)
    cm_performance['final_incentive'] = cm_performance['final_incentive'].round(2)

    # --- 6. Final Report Generation ---
    print("Generating the final incentive report...")

    # Select and reorder columns for the final report
    final_report = cm_performance[[
        'cm_code',
        'employee_name',
        'branch',
        'state_name',
        'monthly_salary',
        'quarterly_salary',
        'total_applications',
        'l2d_tat_score',
        'l2s_tat_score',
        'final_tat_score',
        'business_score',
        'portfolio_score',
        'overall_score',
        'eligible_incentive',
        'final_incentive'
    ]]

    print("--- Incentive Calculation Complete ---")
    return final_report

## Testing

In [7]:
disbursal_data_q3 = {
    'application_id': range(101, 141),
    'disb_month': ['July 2025']*10 + ['August 2025']*15 + ['September 2025']*15,
    'branch': ['Mumbai']*12 + ['Ahmedabad']*15 + ['Pune']*13,
    'state_name': ['Maharashtra']*12 + ['Gujarat']*15 + ['Maharashtra']*13,
    # Login to Disbursement TAT
    'l2d_tat': [3, 5, 4, 6, 2, 8, 4, 5, 3, 6, 7, 4,
                8, 10, 9, 12, 7, 6, 9, 11, 8, 10, 13, 9, 8, 7, 6,
                2, 4, 3, 1, 5, 6, 2, 4, 3, 2, 5, 6, 1],
    # Login to Sanction TAT
    'l2s_tat': [2, 4, 3, 5, 1, 7, 3, 4, 2, 5, 6, 3,
                7, 9, 8, 11, 6, 5, 8, 10, 7, 9, 12, 8, 7, 6, 5,
                1, 3, 2, 1, 4, 5, 1, 3, 2, 1, 4, 5, 1]
}
disbursal_df_q3 = pd.DataFrame(disbursal_data_q3)

# Sample Employee Data for Credit Managers
employee_data_cm = {
    'cm_code': ['CM201', 'CM202', 'CM203'],
    'employee_name': ['Vikram Rathod', 'Anjali Desai', 'Sanjay Kapoor'],
    'designation': ['Credit Manager', 'Credit Manager', 'Credit Manager'],
    'branch': ['Mumbai', 'Ahmedabad', 'Pune'],
    'state_name': ['Maharashtra', 'Gujarat', 'Maharashtra'],
    'monthly_salary': [80000, 85000, 78000]
}
employee_df_cm = pd.DataFrame(employee_data_cm)


# --- Execution ---
# Set the reporting quarter
reporting_quarter = ['July 2025', 'August 2025', 'September 2025']

# Calculate the incentive
cm_incentive_report = calculate_cm_incentive(disbursal_df_q3, employee_df_cm, reporting_quarter)

# Display the final report
if not cm_incentive_report.empty:
    print("\n--- Credit Manager Quarterly Incentive Report ---")
    print(cm_incentive_report.to_string())

--- Starting Incentive Calculation for Quarter: July 2025, August 2025, September 2025 ---
Calculating TAT Score for each Branch/Credit Manager...
Calculating Business Score...
Applying placeholder for Portfolio Score...
Calculating final incentive...
Generating the final incentive report...
--- Incentive Calculation Complete ---

--- Credit Manager Quarterly Incentive Report ---
  cm_code  employee_name     branch   state_name  monthly_salary  quarterly_salary  total_applications  l2d_tat_score  l2s_tat_score  final_tat_score  business_score  portfolio_score  overall_score  eligible_incentive  final_incentive
0   CM201  Vikram Rathod     Mumbai  Maharashtra           80000            240000                  12             50             50             50.0              50                0           30.0             60000.0          18000.0
1   CM202   Anjali Desai  Ahmedabad      Gujarat           85000            255000                  15             75             75             75